In [ ]:
import requests
import json
import os
import pandas as pd
import random

In [ ]:
baseURL="https://backend.libtech.in"
baseURL = "http://backend.libtech.in:8080"
AUTHENDPOINT='%s/api/user/token/' % (baseURL)
TESTURL="%s/api/test/" % (baseURL)
REPORTURL="%s/api/public/report/" % (baseURL)
TAGLOCATIONSURL = "%s/api/public/taglocations/" % (baseURL)
TASKURL="%s/api/public/queue/" % (baseURL)
GETREPORTURL="%s/api/getReport/" % (baseURL)
LOCATIONURL="%s/api/public/location/" % (baseURL)
bundle_endpoint = "%s/api/public/bundle/" % (baseURL)


In [ ]:
def get_headers(email, password):
    data={
      'email' : email,
      'password' : password
            }
    r=requests.post(AUTHENDPOINT,data=data)
    token=r.json()['access']
    headers={
      'content-type':'application/json',
      "Authorization" : "Bearer " + token
    }
    return headers

In [ ]:
email = os.getenv("LIBTECH_API_USERNAME")
password = os.getenv("LIBTECH_API_PASSWORD")
headers = get_headers(email, password)
headers

## Getting all Orissa Blocks

In [ ]:
url = f"{REPORTURL}?location_type=state&report_type=nrega_locations&location_code=24"
print(url)
r = requests.get(url, headers=headers)
res = r.json()
results = res["results"]
report_url = results[0].get("report_url")
print(report_url)

## Getting all the block codes

In [ ]:
dataframe = pd.read_csv(report_url, dtype={'block_code': object})
all_block_codes = dataframe.block_code.unique().tolist()
print(len(all_block_codes))

## Sampling the required Blocks

In [ ]:
to_be_sampled = 74
sampled_blocks = random.sample(all_block_codes, to_be_sampled)
print(len(sampled_blocks))

## Setting up data for tag post request
### if recursive is set to True, it would also tag the child locations. In the above case it would also tag all the panchayats with this code

In [ ]:
tag_name = "test"
data = {
    "tag" : tag_name,
    "locations" : sampled_blocks,
    "recursive" : False
}

In [ ]:
r = requests.post(TAGLOCATIONSURL, headers= headers, data=json.dumps(data))
print(r.status_code)
print(r.content)